In [ ]:
'''
%%writefile gap.py
start = time.time()
end = time.time()
print(end-start)
'''
import tensorflow as tf
import tensorflow_probability as tfp
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import os as os
import sys
from pathlib import Path

mhsdir = Path(os.getcwd()).parent
sys.path.append(os.path.join(mhsdir, 'src'))

import ns
import gap

datarawdir = os.path.join(mhsdir, 'data\\raw\\YC')
dataprepdir = os.path.join(mhsdir, 'data\\preprocessed')
dataprepnsdir = os.path.join(mhsdir, 'data\\preprocessed\\ns')

tempdatarawdir = os.path.join(mhsdir, 'temp\\data\\raw\\YC')
tempdataprepdir = os.path.join(mhsdir, 'temp\\data\\preprocessed')
tempdataprepnsdir = os.path.join(mhsdir, 'temp\\data\\preprocessed\\ns')

tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers
tfb = tfp.bijectors

#!pip install -q requests
import requests
import json
import tempfile
modeldir = os.path.join(mhsdir, 'models')

# Autoregressive model

In [ ]:
model = tfk.Sequential([
    # NOTE: This model takes no input and outputs a Distribution.  (We use
    # the batch_size and type of the input, but there are no actual input
    # values because the last dimension of the shape is 0.)
    #
    # For conditional density estimation, the model would take the
    # conditioning values as input.)
    tfk.layers.InputLayer(input_shape=(0,), dtype=tf.float32),

    # Given the empty input, return a standard normal distribution with
    # matching batch_shape and event_shape of [2].
    tfpl.DistributionLambda(lambda t: tfd.MultivariateNormalDiag(
        
        loc=tf.zeros(tf.concat([tf.shape(t)[:-1], [4]], axis=0)),
        scale_diag=[1., 1., 1., 1.])),

    # Transform the standard normal distribution with event_shape of [2] to
    # the target distribution with event_shape of [2].
    tfpl.AutoregressiveTransform(tfb.AutoregressiveNetwork(
        params=2, hidden_units=[32], activation='relu')),
])

model.compile(
    optimizer=tfk.optimizers.Adam(),
    loss=lambda y, rv_y: -rv_y.log_prob(y))

In [ ]:
y = iter(gap.make_dataset(tempdataprepnsdir, eps = 0.00001).batch(640000).take(1)).next()
x = np.zeros((y.shape[0], 0))
hist = model.fit(x=x,
          y=y,
          batch_size=32,
          epochs=32,
          verbose=True)

In [ ]:
# Use the fitted distribution.
distribution = model(np.zeros((0,)))
distribution.sample(4)
distribution.log_prob(np.zeros((5, 3, 2)))

# Generate data from GAP and Autoregressive

In [ ]:
model_gap = tf.keras.models.load_model("ES4_GW32_GD4_DW32_DD4_BS64/generator")
model_autoreg = model(np.zeros((0,)))

In [ ]:
ActTerms = ['D1', 'M01', 'M03', 'M04', 'M06', 'M07', 'M09', 
            'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 'Y09', 'Y10', 'Y12', 'Y15', 'Y20', 'Y30']

@tf.function
def transform_to_yc(x):
    terms = tf.constant([[0.033333, 1.0, 3.0, 4.0, 6.0, 7.0, 9.0,
                          12.0, 24.0, 36.0, 48.0, 60.0, 72.0, 84.0, 96.0, 108.0, 120.0, 144.0, 180.0, 240.0, 360.0, 600.]],dtype = tf.float32)   
    batch = tf.shape(x)[0]
    sz = tf.constant(terms.shape[1])
    val1 = tf.divide(terms, tf.slice(x, [0,0], [batch,1]))
    val2 = tf.math.exp(-val1)
    val3 = tf.divide(1.0 - val2, val1)
    y = tf.add(tf.concat([tf.ones([batch,sz,1]), tf.zeros([batch,sz,1]), tf.expand_dims(-val2, 2)], axis = -1),
               tf.math.multiply(tf.expand_dims(val3, 2), [[-1.0, 1.0, 1.0]]))
    return  tf.einsum('...ik,...k->...i',y, tf.slice(x, [0 , 1],[batch, 3]))

def generate_param_gap(model, cnt, thau_min = 10.0, thau_max = 150.0, max_dev = -0.1): 
    x = tf.repeat(tf.constant([[0.0, 1.0]]), cnt, axis = 0)
    mm = model(x, training=False).numpy()
    lx = (mm[:,0] > thau_min) & (mm[:,0] < thau_max) 
    mm = mm[lx]
    yy = transform_to_yc(mm).numpy()
    max_y = np.max(yy, axis = 1) 
    deviation = (yy[:,-1] - max_y) / max_y
    yy = yy[deviation > max_dev]
    return yy[:, :-1]

def generate_param_autoregressive(model, cnt, thau_min = 10.0, thau_max = 150.0, max_dev = -0.1): 
    mm = model.sample(cnt).numpy()
    lx = (mm[:,0] > thau_min) & (mm[:,0] < thau_max) & (mm[:,1] > 0.0001) & (mm[:,2] > 0.0001)
    mm = mm[lx]
    yy = transform_to_yc(mm).numpy()
    max_y = np.max(yy, axis = 1) 
    deviation = (yy[:,-1] - max_y) / max_y
    yy = yy[deviation > max_dev]
    return yy[:, :-1]

def plot_net(ym):
    x = np.array([0.033333, 1.0, 3.0, 4.0, 6.0, 7.0, 9.0,12.0, 24.0, 36.0, 48.0, 60.0, 72.0, 84.0, 96.0, 108.0,
                  120.0, 144.0, 180.0, 240.0, 360.0])
    num_components =64
    fig = plt.figure(figsize = (16,16))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    for i in range(num_components):
        y = ym[i,:]
        ax = fig.add_subplot(8,8,1+i)
        ax.plot(x, y)

In [ ]:
ym_gap = generate_param_gap(model = model_gap, cnt = 64000 * 4, thau_min = 10.0, thau_max = 150.0, max_dev = -0.1)
ym_autreg = generate_param_autoregressive(model = model_autoreg, cnt = 64000 * 4, thau_min = 10.0, thau_max = 150.0, max_dev = -0.1)
df1 = pd.DataFrame( data = ym_gap, columns = ActTerms)
df2 = pd.DataFrame(data = ym_autreg, columns = ActTerms)
df = df1.append(df2)
for _ in range(100):
    df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.to_csv(os.path.join(dataprepnsdir, 'Generated_Total_225492.csv'), index = False)

In [ ]:
ym_gap = generate_param_gap(model = model_gap, cnt = 64, thau_min = 10.0, thau_max = 150.0, max_dev = -0.1)
plot_net(ym_gap)

In [ ]:
ym_autreg = generate_param_autoregressive(model = model_autoreg, cnt = 64, thau_min = 10.0, thau_max = 150.0, max_dev = -0.1)
plot_net(ym_autreg)